In [1]:
# # E-commerce Dashboard: Czyszczenie danych
# 
# Ten notebook zawiera proces czyszczenia i przygotowania danych do dalszej analizy.

# ## 1. Import bibliotek

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime

# Ustawienia wizualizacji
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
sns.set_palette('viridis')
%matplotlib inline

In [3]:
# ## 2. Wczytanie danych

# Ścieżka do danych
data_path = '../data/raw/brazilian-ecommerce/'

# Wczytywanie plików CSV
orders = pd.read_csv(f"{data_path}olist_orders_dataset.csv")
order_items = pd.read_csv(f"{data_path}olist_order_items_dataset.csv")
products = pd.read_csv(f"{data_path}olist_products_dataset.csv")
customers = pd.read_csv(f"{data_path}olist_customers_dataset.csv")
geolocation = pd.read_csv(f"{data_path}olist_geolocation_dataset.csv", low_memory=False)
payments = pd.read_csv(f"{data_path}olist_order_payments_dataset.csv")
reviews = pd.read_csv(f"{data_path}olist_order_reviews_dataset.csv")
category_translation = pd.read_csv(f"{data_path}product_category_name_translation.csv")


In [5]:
# ## 3. Konwersja kolumn dat na format datetime

print("Konwersja dat w tabeli zamówień...")

# Kolumny dat w tabeli zamówień
date_columns = [
    'order_purchase_timestamp', 
    'order_approved_at', 
    'order_delivered_carrier_date', 
    'order_delivered_customer_date', 
    'order_estimated_delivery_date'
]

for col in date_columns:
    orders[col] = pd.to_datetime(orders[col], errors='coerce')

# Sprawdzamy, jak wygląda tabela po konwersji
print("\nTabela zamówień po konwersji dat:")
orders.info()

# Kolumny dat w tabeli recenzji
reviews['review_creation_date'] = pd.to_datetime(reviews['review_creation_date'])
reviews['review_answer_timestamp'] = pd.to_datetime(reviews['review_answer_timestamp'])


Konwersja dat w tabeli zamówień...

Tabela zamówień po konwersji dat:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97658 non-null  datetime64[ns]
 6   order_delivered_customer_date  96476 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
dtypes: datetime64[ns](5), object(3)
memory usage: 6.1+ MB


In [7]:
# ## 4. Tworzenie dodatkowych kolumn z datą

print("Dodawanie kolumn z datą...")

# Dodajemy kolumny z rokiem, miesiącem, dniem, dniem tygodnia i godziną
orders['purchase_year'] = orders['order_purchase_timestamp'].dt.year
orders['purchase_month'] = orders['order_purchase_timestamp'].dt.month
orders['purchase_day'] = orders['order_purchase_timestamp'].dt.day
orders['purchase_dayofweek'] = orders['order_purchase_timestamp'].dt.dayofweek
orders['purchase_hour'] = orders['order_purchase_timestamp'].dt.hour

# Dodajemy kolumnę yearmonth dla późniejszej agregacji miesięcznej
orders['purchase_yearmonth'] = orders['order_purchase_timestamp'].dt.strftime('%Y-%m')

# Dodajemy kolumnę z czasem dostawy (w dniach)
orders['delivery_time'] = (orders['order_delivered_customer_date'] - orders['order_purchase_timestamp']).dt.days

# Dodajemy kolumnę ze statusem dostawy (na czas / opóźniona)
orders['delivery_status'] = np.where(
    orders['order_delivered_customer_date'] <= orders['order_estimated_delivery_date'], 
    'on_time', 
    'delayed'
)

# Sprawdzamy nowe kolumny
print("\nNowe kolumny w tabeli zamówień:")
print(orders[['purchase_year', 'purchase_month', 'purchase_dayofweek', 'purchase_hour', 
             'purchase_yearmonth', 'delivery_time', 'delivery_status']].head())

Dodawanie kolumn z datą...

Nowe kolumny w tabeli zamówień:
   purchase_year  purchase_month  purchase_dayofweek  purchase_hour  \
0           2017              10                   0             10   
1           2018               7                   1             20   
2           2018               8                   2              8   
3           2017              11                   5             19   
4           2018               2                   1             21   

  purchase_yearmonth  delivery_time delivery_status  
0            2017-10            8.0         on_time  
1            2018-07           13.0         on_time  
2            2018-08            9.0         on_time  
3            2017-11           13.0         on_time  
4            2018-02            2.0         on_time  


In [9]:
# ## 5. Łączenie tabel

# ### 5.1 Łączenie tabeli kategorii produktów z tłumaczeniami

print("Łączenie tabeli produktów z tłumaczeniami kategorii...")

# Łączymy tabelę produktów z tłumaczeniami kategorii
products = products.merge(
    category_translation, 
    on='product_category_name', 
    how='left'
)

print("\nTabela produktów po połączeniu z tłumaczeniami:")
products.head()

# ### 5.2 Łączenie danych o zamówieniach z danymi o produktach

print("Łączenie danych o zamówieniach z danymi o produktach...")

# Łączymy dane o elementach zamówień z danymi o produktach
order_products = order_items.merge(
    products, 
    on='product_id', 
    how='left'
)

print("\nPołączone dane o zamówieniach i produktach:")
order_products.head()

# ### 5.3 Łączenie danych o płatnościach z zamówieniami

print("Agregacja danych o płatnościach...")

# Agregujemy dane o płatnościach na poziomie zamówienia
order_payments = payments.groupby('order_id').agg({
    'payment_value': 'sum',
    'payment_installments': 'sum',
    'payment_type': lambda x: x.value_counts().index[0]  # najczęstszy typ płatności
}).reset_index()

print("\nZagregowane dane o płatnościach:")
order_payments.head()

# ### 5.4 Łączenie wszystkich danych w jedną tabelę

print("Łączenie wszystkich danych w jedną tabelę...")

# Łączymy dane o zamówieniach z danymi o płatnościach
orders_full = orders.merge(
    order_payments, 
    on='order_id', 
    how='left'
)

# Łączymy dane z danymi o klientach
orders_full = orders_full.merge(
    customers[['customer_id', 'customer_unique_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state']], 
    on='customer_id', 
    how='left'
)

print("\nPełna tabela zamówień:")
orders_full.head()

Łączenie tabeli produktów z tłumaczeniami kategorii...

Tabela produktów po połączeniu z tłumaczeniami:
Łączenie danych o zamówieniach z danymi o produktach...

Połączone dane o zamówieniach i produktach:
Agregacja danych o płatnościach...

Zagregowane dane o płatnościach:
Łączenie wszystkich danych w jedną tabelę...

Pełna tabela zamówień:


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,purchase_year,purchase_month,...,purchase_yearmonth,delivery_time,delivery_status,payment_value,payment_installments,payment_type,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,2017,10,...,2017-10,8.0,on_time,38.71,3.0,voucher,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,2018,7,...,2018-07,13.0,on_time,141.46,1.0,boleto,af07308b275d755c9edb36a90c618231,47813,barreiras,BA
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,2018,8,...,2018-08,9.0,on_time,179.12,3.0,credit_card,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,2017,11,...,2017-11,13.0,on_time,72.20,1.0,credit_card,7c142cf63193a1473d2e66489a9ae977,59296,sao goncalo do amarante,RN
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,2018,2,...,2018-02,2.0,on_time,28.62,1.0,credit_card,72632f0f9dd73dfee390c9b22eb56dd6,9195,santo andre,SP


In [10]:
# ## 6. Obsługa brakujących wartości

print("Sprawdzanie brakujących wartości w pełnej tabeli zamówień...")
missing_values = orders_full.isnull().sum()
print(missing_values[missing_values > 0])

# ### 6.1 Analiza brakujących dat dostawy

print("\nAnaliza brakujących dat dostawy...")
print(f"Liczba zamówień bez daty dostawy: {orders_full['order_delivered_customer_date'].isnull().sum()}")
print(f"Procentowy udział: {orders_full['order_delivered_customer_date'].isnull().mean() * 100:.2f}%")

# Sprawdzamy statusy zamówień bez daty dostawy
print("\nStatusy zamówień bez daty dostawy:")
orders_full[orders_full['order_delivered_customer_date'].isnull()]['order_status'].value_counts()

# ### 6.2 Obsługa brakujących wartości

print("\nObsługa brakujących wartości...")

# Tworzymy kopię danych
orders_clean = orders_full.copy()

# Usuwamy brakujące daty dostawy tylko dla zamówień ze statusem "delivered"
mask_delivered_no_date = (orders_clean['order_status'] == 'delivered') & (orders_clean['order_delivered_customer_date'].isnull())
print(f"Liczba dostarczonych zamówień bez daty dostawy: {mask_delivered_no_date.sum()}")

# Uzupełniamy brakujące czasy dostawy
orders_clean['delivery_time'] = orders_clean['delivery_time'].fillna(-1)

# Uzupełniamy brakujące statusy dostawy
orders_clean['delivery_status'] = orders_clean['delivery_status'].fillna('unknown')

# Sprawdzamy pozostałe brakujące wartości
print("\nPozostałe brakujące wartości:")
missing_values_clean = orders_clean.isnull().sum()
print(missing_values_clean[missing_values_clean > 0])


Sprawdzanie brakujących wartości w pełnej tabeli zamówień...
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
delivery_time                    2965
payment_value                       1
payment_installments                1
payment_type                        1
dtype: int64

Analiza brakujących dat dostawy...
Liczba zamówień bez daty dostawy: 2965
Procentowy udział: 2.98%

Statusy zamówień bez daty dostawy:

Obsługa brakujących wartości...
Liczba dostarczonych zamówień bez daty dostawy: 8

Pozostałe brakujące wartości:
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
payment_value                       1
payment_installments                1
payment_type                        1
dtype: int64


In [11]:
# ## 7. Zapisywanie przetworzonych danych

# Tworzymy katalog na przetworzone dane
processed_path = '../data/processed/'
os.makedirs(processed_path, exist_ok=True)

print("Zapisywanie przetworzonych danych...")

# Zapisujemy przetworzone tabele
orders_clean.to_csv(f"{processed_path}orders_clean.csv", index=False)
order_products.to_csv(f"{processed_path}order_products.csv", index=False)

print("Zapisane pliki:")
print(f"1. {processed_path}orders_clean.csv - Oczyszczone i połączone dane o zamówieniach")
print(f"2. {processed_path}order_products.csv - Dane o produktach w zamówieniach")

Zapisywanie przetworzonych danych...
Zapisane pliki:
1. ../data/processed/orders_clean.csv - Oczyszczone i połączone dane o zamówieniach
2. ../data/processed/order_products.csv - Dane o produktach w zamówieniach


In [15]:
# ## 8. Przygotowanie zagregowanych danych do analizy

# ### 8.1 Agregacja danych na poziomie miesięcznym

print("Przygotowanie danych zagregowanych miesięcznie...")

monthly_orders = orders_clean.groupby('purchase_yearmonth').agg({
    'order_id': 'count',
    'payment_value': 'sum',
    'delivery_time': lambda x: x[x >= 0].mean()  # średni czas dostawy z pominięciem niezdefiniowanych
}).reset_index()

monthly_orders.columns = ['yearmonth', 'order_count', 'total_sales', 'avg_delivery_time']
monthly_orders.sort_values('yearmonth', inplace=True)

print("\nZagregowane dane miesięczne:")
monthly_orders.head()

# Zapisujemy dane zagregowane
monthly_orders.to_csv(f"{processed_path}monthly_orders.csv", index=False)

# ### 8.2 Agregacja danych na poziomie kategorii produktów

print("Przygotowanie danych zagregowanych według kategorii produktów...")

product_categories = order_products.groupby('product_category_name_english').agg({
    'order_id': 'count',
    'price': 'sum',
    'freight_value': 'sum'
}).reset_index()

product_categories.columns = ['category', 'order_count', 'total_sales', 'total_shipping']
product_categories['avg_price'] = product_categories['total_sales'] / product_categories['order_count']
product_categories.sort_values('order_count', ascending=False, inplace=True)

print("\nZagregowane dane według kategorii produktów:")
product_categories.head()

# Zapisujemy dane zagregowane
product_categories.to_csv(f"{processed_path}product_categories.csv", index=False)

# ### 8.3 Agregacja danych na poziomie stanów

print("Przygotowanie danych zagregowanych według stanów...")

state_sales = orders_clean.groupby('customer_state').agg({
    'order_id': 'count',
    'payment_value': 'sum'
}).reset_index()

state_sales.columns = ['state', 'order_count', 'total_sales']
state_sales.sort_values('total_sales', ascending=False, inplace=True)

print("\nZagregowane dane według stanów:")
state_sales.head()

# Zapisujemy dane zagregowane
state_sales.to_csv(f"{processed_path}state_sales.csv", index=False)


Przygotowanie danych zagregowanych miesięcznie...

Zagregowane dane miesięczne:
Przygotowanie danych zagregowanych według kategorii produktów...

Zagregowane dane według kategorii produktów:
Przygotowanie danych zagregowanych według stanów...

Zagregowane dane według stanów:


In [17]:

# ## 9. Podsumowanie

print("\nPodsumowanie procesu czyszczenia danych:")
print("1. Skonwertowano kolumny dat na format datetime")
print("2. Dodano kolumny pomocnicze z rokiem, miesiącem, dniem tygodnia itd.")
print("3. Połączono dane z różnych tabel w jedną spójną tabelę")
print("4. Obsłużono brakujące wartości")
print("5. Przygotowano zagregowane dane do dalszej analizy")
print("6. Zapisano przetworzone dane w katalogu ../data/processed/")

print("\nW następnym notebooku wykorzystamy te dane do analizy sprzedaży.")


Podsumowanie procesu czyszczenia danych:
1. Skonwertowano kolumny dat na format datetime
2. Dodano kolumny pomocnicze z rokiem, miesiącem, dniem tygodnia itd.
3. Połączono dane z różnych tabel w jedną spójną tabelę
4. Obsłużono brakujące wartości
5. Przygotowano zagregowane dane do dalszej analizy
6. Zapisano przetworzone dane w katalogu ../data/processed/

W następnym notebooku wykorzystamy te dane do analizy sprzedaży.
